In [1]:
import ee
import geemap
import os
import calendar

import rasterio
import numpy as np
from PIL import Image

ee.Authenticate()
ee.Initialize()

In [2]:
# ==========================================
# 1. LOAD DHS CLUSTERS FROM ASSET
# ==========================================

# Define asset path for DHS clusters
ASSET_ID = 'projects/integrated-hawk-485001-k3/assets/ph_dhs_sample_50'

# Load the asset directly as a FeatureCollection
dhs_points = ee.FeatureCollection(ASSET_ID)

# Verify the data loaded correctly by printing the size
print(f"Loaded {dhs_points.size().getInfo()} points from asset.")

Loaded 50 points from asset.


In [3]:
# ==========================================
# 2. DEFINE BUFFER FUNCTION
# ==========================================
# We need to turn each single point into a square box for the CNN.
# 2km buffer for urban, 5km for rural

def adaptive_buffer(feature):
    # 1. Get the Urban/Rural flag from column 'URBAN_RURA'
    urban_rural_status = ee.String(feature.get('URBAN_RURA'))
    
    # 2. Check if it is Urban (Compare string to 'U')
    is_urban = urban_rural_status.compareTo('U').eq(0)
    
    # 3. Choose Radius based on status
    # Logic: If Urban -> 2000m, Else -> 5000m
    radius = ee.Number(ee.Algorithms.If(is_urban, 2000, 5000))
    
    # 4. Buffer and Box
    return feature.buffer(radius).bounds()

# Apply the buffer function to all DHS points
dhs_squares = dhs_points.map(adaptive_buffer)
cluster_ids = dhs_squares.aggregate_array('DHSCLUST').getInfo()
tif_filenames = [f"dhs_{str(c_id)}.tif" for c_id in cluster_ids]

# Verify on Map
Map = geemap.Map()
Map.centerObject(dhs_points, 6)
Map.addLayer(dhs_points, {'color': 'red'}, 'Original DHS Points')
Map.addLayer(dhs_squares, {'color': 'blue'}, 'Buffered Squares (Download Area)')
Map

Map(center=[12.15445433292109, 122.42655412583971], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
# ==========================================
# SETUP
# ==========================================
# Change this to your desired output folder
base_output_dir = os.path.expanduser('/Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/sample50-monthly-2022')
YEAR = 2022

if not os.path.exists(base_output_dir):
    os.makedirs(base_output_dir)

In [5]:
# Define the cloud masking function (Standard S2)
def mask_s2_clouds(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask).divide(10000)

In [8]:
# ==========================================
# NESTED LOOP: CLUSTERS -> MONTHS
# ==========================================

# Get the list of features (Clusters)
features_list = dhs_squares.getInfo()['features']
print(f"Found {len(features_list)} clusters. Starting monthly download...")

for i, feature in enumerate(features_list):
    cluster_id = str(feature['properties']['DHSCLUST'])
    
    # 1. Create a dedicated folder for this cluster
    cluster_dir = os.path.join(base_output_dir, cluster_id)
    if not os.path.exists(cluster_dir):
        os.makedirs(cluster_dir)

    # Get the geometry for this specific cluster
    roi_geometry = ee.Geometry.Polygon(feature['geometry']['coordinates'])

    print(f"[{i+1}/{len(features_list)}] Processing Cluster {cluster_id}...")

    # 2. Loop through 12 months
    for month in range(1, 13):
        # Calculate start and end dates for the month
        _, last_day = calendar.monthrange(YEAR, month)
        start_date = f'{YEAR}-{month:02d}-01'
        end_date = f'{YEAR}-{month:02d}-{last_day}'
        
        filename = f"dhs_{cluster_id}_{YEAR}_{month:02d}.tif"
        out_path = os.path.join(cluster_dir, filename)

        # Skip if already downloaded
        if os.path.exists(out_path):
            continue

        try:
            # 1. LOAD COLLECTION (Do NOT select bands yet!)
            monthly_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterBounds(roi_geometry)
                .filterDate(start_date, end_date)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100)))

            if monthly_col.size().getInfo() > 0:
                # 2. CREATE LAYERS
                
                # Best Case: Apply Cloud Mask FIRST, THEN select bands
                best_layer = (monthly_col
                              .map(mask_s2_clouds)
                              .select(['B4', 'B3', 'B2', 'B8']) # <--- Select happens HERE now
                              .median())
                
                # Backup Case: Raw Mosaic (No cloud mask), THEN select bands
                backup_layer = (monthly_col
                                .select(['B4', 'B3', 'B2', 'B8'])
                                .mosaic()
                                .divide(10000))
                
                # 3. COMBINE & CLIP
                # Use unmask to fill holes in the Best Layer with the Backup Layer
                final_img = best_layer.unmask(backup_layer).clip(roi_geometry)

                # 4. DOWNLOAD
                geemap.download_ee_image(
                    image=final_img,
                    filename=out_path,
                    region=roi_geometry,
                    scale=10,
                    crs='EPSG:3857',
                    overwrite=True
                )
            else:
                print(f"  - Warning: Strictly zero images for Month {month:02d}")
        except Exception as e:
            print(f"  - Error downloading {month:02d}: {e}")

print("Monthly download complete.")

Found 50 clusters. Starting monthly download...
[1/50] Processing Cluster 441...
[2/50] Processing Cluster 450...
[3/50] Processing Cluster 442...
[4/50] Processing Cluster 491...
[5/50] Processing Cluster 512...
[6/50] Processing Cluster 558...
[7/50] Processing Cluster 563...
[8/50] Processing Cluster 600...
[9/50] Processing Cluster 532...
[10/50] Processing Cluster 651...
[11/50] Processing Cluster 638...
[12/50] Processing Cluster 667...
[13/50] Processing Cluster 699...
[14/50] Processing Cluster 727...
[15/50] Processing Cluster 705...


  0%|          |0/12 tiles [00:00<?]

  0%|          |0/12 tiles [00:00<?]

  0%|          |0/12 tiles [00:00<?]

  0%|          |0/12 tiles [00:00<?]

...

  0%|          |0/4 tiles [00:00<?]

  0%|          |0/4 tiles [00:00<?]

  0%|          |0/4 tiles [00:00<?]

  0%|          |0/4 tiles [00:00<?]

Monthly download complete.


In [10]:
def check_missing_months(base_dir, year=2022):
    """
    Scans the output directory to identify clusters with missing monthly images.
    
    Args:
        base_dir (str): Path to the folder containing cluster subfolders.
        year (int): The year to check (default 2023).
    """
    print(f"Checking dataset in: {base_dir} for Year {year}...\n")
    
    # Get all cluster folders (ignoring hidden files like .DS_Store)
    cluster_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
    
    # Sort them for clean output (numerical sort if possible)
    try:
        cluster_folders.sort(key=lambda x: int(x))
    except ValueError:
        cluster_folders.sort()

    missing_log = {}
    total_clusters = len(cluster_folders)
    complete_count = 0

    for cluster_id in cluster_folders:
        cluster_path = os.path.join(base_dir, cluster_id)
        missing_months = []

        # Check months 01 to 12
        for month in range(1, 13):
            # The expected filename format from your previous code
            expected_filename = f"dhs_{cluster_id}_{year}_{month:02d}.tif"
            file_path = os.path.join(cluster_path, expected_filename)
            
            if not os.path.exists(file_path):
                missing_months.append(month)

        # Log results
        if missing_months:
            missing_log[cluster_id] = missing_months
            # Print immediately so you see it while running
            print(f"⚠️ Cluster {cluster_id}: Missing {len(missing_months)} images -> Months {missing_months}")
        else:
            complete_count += 1

    # Final Summary
    print("-" * 40)
    print(f"SUMMARY REPORT")
    print("-" * 40)
    print(f"Total Clusters Scanned: {total_clusters}")
    print(f"Complete Clusters (12/12): {complete_count}")
    print(f"Incomplete Clusters: {len(missing_log)}")
    
    if len(missing_log) == 0:
        print("\n✅ SUCCESS: All clusters have complete monthly data!")
    else:
        print("\n❌ ACTION REQUIRED: Re-run the download script for the missing months.")

# ==========================================
# RUN THE CHECK
# ==========================================
# Make sure this matches your actual output path
output_folder = os.path.expanduser('/Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/sample50-monthly-2022')

check_missing_months(output_folder, year=2022)

Checking dataset in: /Users/ruben/Desktop/Thesis/TrainingData/Sentinel2/sample50-monthly-2022 for Year 2022...

----------------------------------------
SUMMARY REPORT
----------------------------------------
Total Clusters Scanned: 50
Complete Clusters (12/12): 50
Incomplete Clusters: 0

✅ SUCCESS: All clusters have complete monthly data!
